# Example of using LightPipelines on Spark OCR
## Install spark-ocr python packge
Need specify path to `spark-ocr-assembly-[version].jar` or `secret`

In [1]:
license = ""
secret = ""
nlp_secret = ""
aws_access_key = ""
aws_secret_key = ""

version = secret.split("-")[0]
spark_ocr_jar_path = "../../target/scala-2.12"

In [ ]:
#%pip install ../../python/dist/spark-ocr-4.2.2.tar.gz

## Initialization of spark session

In [ ]:
import sys
import os
from pyspark.sql import SparkSession
from sparkocr import start

if license:
    os.environ['SPARK_OCR_LICENSE'] = license
    

spark = start(secret=secret,
              jar_path=spark_ocr_jar_path)

In [7]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import *
from sparkocr.transformers import *

In [8]:
import pkg_resources
pdfs_path = pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/multiplepages')


## Define OCR pipeline

In [31]:
# Transform binary to image

pdf_to_text = PdfToText()
pdf_to_text.setOutputCol("text")
pdf_to_text.setStoreSplittedPdf(True)

pdf_to_image = PdfToImage()
pdf_to_image.setOutputCol("image")

skew_corrector = ImageSkewCorrector()
skew_corrector.setOutputCol("corrected_image")

# Run OCR for each region
ocr = ImageToText()
ocr.setInputCol("corrected_image")
ocr.setOutputCol("text")
ocr.setConfidenceThreshold(60)

# OCR pipeline
pipeline = PipelineModel(stages=[
    pdf_to_text,
    pdf_to_image,
    skew_corrector,
    ocr
])

## Create LightPipeline

In [10]:
from sparkocr.base import LightPipeline

In [32]:
lp = LightPipeline(pipeline)

In [35]:
%%time
import pandas as pd
import itertools as it
pd.DataFrame(lp.fromLocalPath(pdfs_path))

CPU times: user 9.56 ms, sys: 4.12 ms, total: 13.7 ms
Wall time: 3.15 s


,text
0,"[Annotation(ImageToText, 0, 16, This is a page..."
1,"[Annotation(ImageToText, 0, 22, This is anothe..."
2,"[Annotation(ImageToText, 0, 18, Yet another pa..."
3,"[Annotation(ImageToText, 0, 16, This is a page..."
4,"[Annotation(ImageToText, 0, 22, This is anothe..."
5,"[Annotation(ImageToText, 0, 18, Yet another pa..."
